# Initialization

In [3]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [5]:
items = pd.read_parquet("items.parquet")
events = pd.read_parquet("events.parquet")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [7]:
# Зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

# Разбиваем данные по дате
train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx].copy()
events_test = events[~train_test_global_time_split_idx].copy()

# Получаем уникальных пользователей в каждой выборке
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()

# Находим пользователей, присутствующих в обеих выборках
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [8]:
# Уникальные пользователи в train и test
users_train = events_train["user_id"].unique()
users_test = events_test["user_id"].unique()

# Находим пересечение (общих пользователей)
common_users = set(users_train) & set(users_test)
num_common_users = len(common_users)

print(f"Количество общих пользователей: {num_common_users}")

Количество общих пользователей: 120858


«Холодные» пользователи — те, которые есть в test, но отсутствуют в train. Это соответствует хронологическому порядку, в котором и работает рекомендательная система.

In [9]:
# Уникальные пользователи в train и test
users_train = set(events_train["user_id"].unique())
users_test = set(events_test["user_id"].unique())

# Холодные пользователи (есть в test, но нет в train)
cold_users = users_test - users_train

print(f"Количество холодных пользователей: {len(cold_users)}")

Количество холодных пользователей: 2365


In [10]:
from sklearn.preprocessing import MinMaxScaler

# Фильтруем события с 2015 года
top_pop_start_date = pd.to_datetime("2015-01-01").date()

# Считаем популярность (уникальные пользователи) и средний рейтинг для каждой книги
item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(
        users=("user_id", "nunique"), 
        avg_rating=("rating", "mean")
    ).reset_index()

# Нормализуем показатели (приводим к диапазону [0, 1])
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# Вычисляем итоговый скор (произведение нормализованных значений)
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# Сортируем по убыванию popularity_score
item_popularity = item_popularity.sort_values("popularity_score", ascending=False)

# Выбираем топ-100 книг со средним рейтингом >= 4
top_k_pop_items = item_popularity[
    item_popularity["avg_rating"] >= 4
].head(100)

# Проверяем результат
print(f"Топ-100 популярных книг с 2015 года (avg_rating >= 4):")
print(top_k_pop_items[["item_id", "users", "avg_rating", "popularity_score"]].head())

Топ-100 популярных книг с 2015 года (avg_rating >= 4):
        item_id  users  avg_rating  popularity_score
32387  18007564  20207    4.321275          0.412333
32623  18143977  19462    4.290669          0.393471
2             3  15139    4.706057          0.344702
30695  16096824  16770    4.301014          0.340108
1916      15881  13043    4.632447          0.291076


### default recommendations

In [11]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
5,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,0.290194,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,0.280247,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,0.279094,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,0.275401,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
9,13206900,Marissa Meyer,"Winter (The Lunar Chronicles, #4)",2015,12291,4.534293,0.266881,"{'Fantasy': 4835, 'Young Adult': 4672, 'Scienc..."


In [12]:
# Объединяем тестовые события холодных пользователей с рейтингами из топ-100
cold_users_events_with_recs = (
    events_test[events_test["user_id"].isin(cold_users)]
    .merge(
        top_k_pop_items[["item_id", "avg_rating"]],  # Берем только нужные столбцы
        on="item_id",
        how="left"
    )
)

# Находим события, для которых нет avg_rating (книга не в топ-100)
cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()

# Отбираем только события с известным avg_rating (книги из топ-100)
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx][
    ["user_id", "item_id", "rating", "avg_rating"]
]

In [13]:
print(f"Всего событий холодных пользователей: {len(cold_users_events_with_recs)}")
print(f"Событий с книгами из топ-100: {len(cold_user_recs)}")
print(f"Пример записей:\n{cold_user_recs.head(10)}")

Всего событий холодных пользователей: 9672
Событий с книгами из топ-100: 1912
Пример записей:
    user_id   item_id  rating  avg_rating
2   1361610  25899336       4    4.427261
5   1338996  16096824       5    4.301014
8   1338996  18692431       5    4.071454
9   1338996  28763485       2    4.194663
15  1276025     38447       5    4.232770
45  1027534     38447       5    4.232770
52  1408652  23437156       5    4.512300
56  1192858  20613470       4    4.495608
57  1192858  28260587       5    4.557174
58  1192858  18006496       5    4.610097


In [14]:
# Общее количество событий холодных пользователей
total_cold_events = len(cold_users_events_with_recs)

# Количество событий с книгами из топ-100 (где avg_rating не NaN)
matched_events = len(cold_user_recs)

# Вычисляем долю и округляем до сотых
match_ratio = round(matched_events / total_cold_events, 2)

print(f"Доля совпадений: {match_ratio}")

Доля совпадений: 0.2


In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Вычисляем RMSE (Root Mean Squared Error)
rmse = mean_squared_error(
    cold_user_recs["rating"], 
    cold_user_recs["avg_rating"], 
    squared=False  # Возвращаем квадратный корень
)

# Вычисляем MAE (Mean Absolute Error)
mae = mean_absolute_error(
    cold_user_recs["rating"], 
    cold_user_recs["avg_rating"]
)

# Выводим результаты с округлением
print(f"RMSE: {round(rmse, 2)}, MAE: {round(mae, 2)}")

RMSE: 0.78, MAE: 0.62


In [16]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

In [17]:
# Уникальные пользователи и предметы
n_users = events['user_id'].nunique()
n_items = events['item_id'].nunique()
total_cells = n_users * n_items

# Уникальные взаимодействия (пары user-item)
filled_cells = len(events[['user_id', 'item_id']].drop_duplicates())

# Степень разреженности
sparsity = 1 - (filled_cells / total_cells)

print(f"Разреженность U-I матрицы: {sparsity:.2%}")

Разреженность U-I матрицы: 99.93%


In [18]:
print(f"Всего пользователей: {n_users}")
print(f"Всего предметов: {n_items}")
print(f"Теоретически возможных взаимодействий: {total_cells:,}")
print(f"Фактических взаимодействий: {filled_cells:,}")

Всего пользователей: 430585
Всего предметов: 41673
Теоретически возможных взаимодействий: 17,943,768,705
Фактических взаимодействий: 11,751,086


In [19]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [20]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [21]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059136 0.647437483750257


In [22]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [23]:
random_mae = accuracy.mae(random_predictions)

MAE:  1.0018


In [24]:
# Вычисляем на сколько процентов случайная модель хуже
percent_worse = ((random_mae - mae) / mae) * 100

print(f"MAE (SVD): {mae:.2f}")
print(f"MAE (Random): {random_mae:.2f}")
print(f"Случайные рекомендации хуже на: {round(percent_worse)}%")

MAE (SVD): 0.65
MAE (Random): 1.00
Случайные рекомендации хуже на: 55%


Факультативное задание
Задание. Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей. Возьмите небольшое N, например 2–3 пользователя. Получите рекомендации, посчитайте метрики, оцените, как они изменились. Подумайте, с чем могут быть связаны такие изменения.

In [25]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):
    """Возвращает n рекомендаций для user_id
    
    Параметры:
    ----------
    user_id : str/int
        Идентификатор пользователя
    all_items : set/list
        Все возможные item_id в системе
    events : pd.DataFrame
        DataFrame с историей взаимодействий
    model : surprise.prediction_algorithms
        Обученная модель рекомендаций
    include_seen : bool, optional
        Включать ли уже просмотренные товары (по умолчанию True)
    n : int, optional
        Количество рекомендаций (по умолчанию 5)
    
    Возвращает:
    -----------
    pd.DataFrame
        DataFrame с колонками item_id и score, отсортированный по убыванию score
    """
    
    # Получаем список всех уникальных книг
    all_items = set(events['item_id'].unique())
    
    # Обрабатываем флаг include_seen
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # Получаем книги, которые пользователь уже видел
        seen_items = set(events[events['user_id'] == user_id]['item_id'].unique())
        # Оставляем только новые книги
        items_to_predict = list(all_items - seen_items)
    
    # Получаем предсказания для всех рассматриваемых книг
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # Сортируем по убыванию оценки и берем топ-N
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    # Возвращаем DataFrame с рекомендациями
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], 
                       columns=["item_id", "score"])

In [26]:
get_recommendations_svd(1296647, items, events_test, svd_model)


,item_id,score
0,7864312,4.981188
1,25793186,4.912001
2,12174312,4.898052
3,13208,4.894869
4,33353628,4.891661


In [27]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations) 

user_id: 1169023
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
68,Veronica Roth,"Divergent (Divergent, #1)",2014-06-02,2014-06-04,4,"{'Young Adult': 20260, 'Science Fiction-Dystop..."
69,"Gillian Flynn, В. Русанов",Gone Girl,2014-05-27,2014-05-29,5,"{'Fiction': 11773, 'Mystery': 9965, 'Thriller'..."
70,Kathy Reichs,"Death du Jour (Temperance Brennan, #2)",2014-05-24,2014-05-27,4,"{'Mystery': 1206, 'Mystery-Crime': 579, 'Ficti..."
71,Chelsea Cain,"Heartsick (Archie Sheridan & Gretchen Lowell, #1)",2014-05-22,2014-05-22,5,"{'Mystery': 832, 'Thriller': 653, 'Fiction': 4..."
72,"Jussi Adler-Olsen, Lisa Hartford","The Keeper of Lost Causes (Department Q, #1)",2014-05-30,2014-06-02,3,"{'Mystery': 1225, 'Mystery-Crime': 627, 'Ficti..."
73,Gillian Flynn,Dark Places,2014-05-17,2014-05-22,4,"{'Mystery': 4534, 'Fiction': 4055, 'Thriller':..."
74,Audrey Niffenegger,Her Fearful Symmetry,2014-05-05,2014-05-08,2,"{'Fiction': 1984, 'Fantasy': 674, 'Fantasy-Par..."
75,Kathy Reichs,"Déjà Dead (Temperance Brennan, #1)",2014-05-13,2014-05-17,4,"{'Mystery': 2141, 'Fiction': 904, 'Mystery-Cri..."
76,Carolyn Parkhurst,The Dogs of Babel,2014-05-09,2014-05-10,5,"{'Fiction': 522, 'Mystery': 102, 'Animals': 77..."
77,George R.R. Martin,"A Dance with Dragons (A Song of Ice and Fire, #5)",2014-05-04,2014-05-04,5,"{'Fantasy': 22247, 'Fiction': 4512, 'Fantasy-E..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,2199,5,Doris Kearns Goodwin,Team of Rivals: The Political Genius of Abraha...,"{'History': 4174, 'Nonfiction': 2127, 'Biograp..."
1,16255632,5,"David Gaider, Ben Gelinas, Mike Laidlaw, Dave ...",Dragon Age: The World of Thedas Volume 1,"{'Fantasy': 134, 'Games-Video Games': 28, 'Art..."
2,2363958,5,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."
3,22552026,5,Jason Reynolds,Long Way Down,"{'Young Adult': 1871, 'Poetry': 1737, 'Contemp..."
4,29237211,5,"Brian K. Vaughan, Fiona Staples","Saga, Vol. 7 (Saga, #7)","{'Sequential Art-Graphic Novels': 2539, 'Seque..."


Факультативное задание
Добавьте в events события для нового пользователя, как если бы он прочитал те книги, которые нравятся вам. Получите рекомендации для этого пользователя. Оцените, насколько они релевантны вашим интересам.

# === Базовые подходы: коллаборативная фильтрация

In [15]:
import scipy
import sklearn.preprocessing

# Перекодировка пользователей
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# Перекодировка объектов
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])

# Применяем к событиям
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

# Определяем максимальное значение
max_item_enc = events_train["item_id_enc"].max()
print(f"Максимальное значение item_id_enc в events_train: {max_item_enc}")

Максимальное значение item_id_enc в events_train: 43304


In [16]:
n_users = events_train['user_id_enc'].nunique()
n_items = events_train['item_id_enc'].nunique()
total_elements = n_users * n_items
size_gb_int = int(total_elements / (1024 ** 3))

print(f"Размер матрицы: {size_gb_int} GB")

Размер матрицы: 16 GB


In [17]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [18]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3 

0.26370687410235405

In [19]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

100%|██████████| 50/50 [07:25<00:00,  8.90s/it]


In [20]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

Факультативное задание
Используя get_recommendations_als, напишите код, который позволит для случайного пользователя просмотреть рекомендации в удобном формате: 
история с именами авторов и названием книг,
рекомендации с именами авторов и названием книг, seen-признаком (взаимодействовал ли уже пользователь с рекомендованной книгой).
Проанализируйте, релевантны ли рекомендации имеющейся истории.

In [21]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [22]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [23]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")

In [25]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

In [26]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [27]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [28]:
print(ndcg_at_5_scores.mean())

0.975955036457971


Факультативное задание
Оцените, для какой доли пользователей удалось посчитать метрику NDCG.

In [29]:
# Общее количество пользователей в тестовой выборке
total_users = events_test['user_id'].nunique()

# Количество пользователей с рассчитанной NDCG
users_with_ndcg = ndcg_at_5_scores.notna().sum()

# Доля пользователей с валидной NDCG
ndcg_coverage = users_with_ndcg / total_users

print(f"Доля пользователей с рассчитанной NDCG@5: {ndcg_coverage:.2%}")

Доля пользователей с рассчитанной NDCG@5: 13.99%


In [30]:
# Причины пропусков
no_ratings_users = events_test.groupby('user_id')['rating'].count()
print(f"Пользователей с < 2 оценками: {(no_ratings_users < 2).sum() / total_users:.2%}")

Пользователей с < 2 оценками: 36.04%


### Рекомендации I2I
Рекомендации вида I2I (item2item) — это рекомендации похожих объектов. Например, к некоторой книге порекомендовать список похожих книг. Рассматриваемая реализация ALS позволяет получить такие рекомендации при помощи метода `similar_items`

### Рекомендации U2U
Рекомендации вида U2U — это рекомендации похожих пользователей, то есть пользователей со схожими предпочтениями. Такой тип рекомендаций широко используется в социальных сетях. Рассматриваемая реализация ALS позволяет получить такие рекомендации при помощи метода `similar_users`.

# === Базовые подходы: контентные рекомендации

# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков